<a href="https://colab.research.google.com/github/h-razu/air-quality-analysis/blob/main/air_quality_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import the Required Library

In [1]:
import pandas as pd
import numpy as np

### Mount Google Drive and Load the Datasets

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd '/content/drive/MyDrive/datasets/assessment_datasets'

/content/drive/MyDrive/datasets/assessment_datasets


In [6]:
%ls

air_quality_data_combined.csv                 PRSA_Data_Gucheng_20130301-20170228.csv
combined_output.csv                           PRSA_Data_Huairou_20130301-20170228.csv
PRSA_Data_Aotizhongxin_20130301-20170228.csv  PRSA_Data_Nongzhanguan_20130301-20170228.csv
PRSA_Data_Changping_20130301-20170228.csv     PRSA_Data_Shunyi_20130301-20170228.csv
PRSA_Data_Dingling_20130301-20170228.csv      PRSA_Data_Tiantan_20130301-20170228.csv
PRSA_Data_Dongsi_20130301-20170228.csv        PRSA_Data_Wanliu_20130301-20170228.csv
PRSA_Data_Guanyuan_20130301-20170228.csv      PRSA_Data_Wanshouxigong_20130301-20170228.csv


In [4]:
#read the each file and combine them
import glob

# Get all CSV files in the folder
csv_files = glob.glob("/content/drive/MyDrive/datasets/assessment_datasets/*.csv")

# Read and combine all CSV files
combined_df = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)

# Save to a new CSV file
combined_df.to_csv("air_quality_data_combined.csv", index=False)

print("All CSV files have been merged into 'air_quality_data_combined.csv'.")


All CSV files have been merged into 'air_quality_data_combined.csv'.
